In [ ]:
import SpaDecon as spd
import scanpy as sc
import pandas as pd
import numpy as np
from skimage import io
import os

In [ ]:
#Please download the spadecon_tutorial_data folder from https://drive.google.com/drive/folders/1_eBGKMVYh4p1eiHhrJASnqOD9z_CP5e0?usp=sharing
#set working directory to spadecon_tutorial_data using os.chdir()

#Read annotated scRNA-seq GE data (rows = cells, columns = genes, cell types in adata_sc.obs.celltype)
adata_sc = sc.read('bc_sc.h5ad')

#Read SRT GE data (rows = spots, columns = genes)
adata_st = sc.read_10x_h5('V1_Breast_Cancer_Block_A_Section_1_filtered_feature_bc_matrix.h5')
adata_st.var_names_make_unique()

#Read SRT histology image
histology = io.imread("V1_Breast_Cancer_Block_A_Section_1_image.tif")

#Read file with SRT spatial locations
locations = pd.read_csv('tissue_positions_list.csv', header = None, index_col = 0)
locations = locations.loc[adata_st.obs.index]

np.random.seed(10)
clf = spd.SpaDecon()
clf.deconvolution(source_data=adata_sc, target_data=adata_st, histology_image=histology, spatial_locations=locations, technology = 'Visium')

In [ ]:
spadecon_props = clf.props
spadecon_props.to_csv('spadecon_props.csv')